https://www.kaggle.com/dyasin/week24ml-weather-dataset-rattle-package-weatheraus/edit/run/68212956

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import accuracy_score,mean_squared_error,roc_curve,roc_auc_score,classification_report,r2_score,confusion_matrix

from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

# For data visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns; sns.set()
# Plotly for interactive graphics 
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import missingno as msno
# Disabling warnings
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# After performing all data processing and cleaning steps, and after splitting train /test.

# Logistic regression
# Naive Bayes
# K-Nearest Neighbor (KNN)
# Support Vector Mechanism (SVM) Get predictions using Machine Learning models and compare these scores.
# Which of these models is the best?

# DATA READING AND EXPLORING

In [ ]:
data = pd.read_csv("../input/weather-dataset-rattle-package/weatherAUS.csv")
df = data.copy().sample(10000)
df

In [ ]:
df.info()

In [ ]:
pd.DataFrame({"No. of unique values": list(df.nunique())}, index=df.columns)

In [ ]:
for i in df.select_dtypes(include='object'):   
    print(i,'-->',df[i].unique())

In [ ]:
y = len(df[df['RainTomorrow'] == 'Yes'])
n = len(df[df['RainTomorrow'] == 'No'])
print(y,n)

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe().T

In [ ]:
print(df.shape)
print(df.ndim)
print(df.size)

In [ ]:
# Here we can see the variables in the model and set the variables for the model. 
# We can also remove some variables.
df.corr()

# SOME VISUALIZATION

In [ ]:
plt.figure(figsize = (16,6)) 
sns.heatmap(df.corr(),robust=True,fmt='.1g',linewidths=1.3,linecolor = 'gold', annot=True,);

In [ ]:
#VISUALIZATION OF NAN  VALUES
msno.matrix(df)

In [ ]:
#drop missing values in the RainToday and RainTomorrow
df.dropna(subset=['RainToday', 'RainTomorrow'],axis=0,inplace=True)

In [ ]:
sns.countplot(x="RainToday",data=df)

In [ ]:
sns.countplot(x="RainTomorrow",data=df)

# Multivariate imputation
* In multivariate imputation, we use ML Algorithms and before that we need to encode the categorical variables.(Çok değişkenli atamada, ML Algoritmaları kullanıyoruz ve bundan önce kategorik değişkenleri kodlamamız gerekiyor.)

In [ ]:
def summary(df):
    Types = df.dtypes
    Counts = df.apply(lambda x: x.count())
    Min = df.min()
    Max = df.max()
    Uniques = df.apply(lambda x: x.unique().shape[0]) # .shape[0] yazilmaz ise unique olan degerlerin listelerini getirir.
    Nulls = df.apply(lambda x: x.isnull().sum())

    cols = ['Types', 'Counts', 'Uniques', 'Nulls', 'Min', 'Max']
    str = pd.concat([Types, Counts, Uniques, Nulls, Min, Max], axis = 1, sort=True)

    str.columns = cols
    display(str.sort_values(by='Nulls', ascending=False))
    print('__________Data Types__________\n')
    print(str.Types.value_counts())
summary(df)

In [ ]:
# Label encoding for univariate variables.
from sklearn.preprocessing import LabelEncoder

lbe = LabelEncoder()
df["RainToday_label"] = lbe.fit_transform(df["RainToday"])
df["RainTomorrow_label"] = lbe.fit_transform(df["RainTomorrow"])

In [ ]:
# one-hot encoding for variables with more than 2 categories

#drop variables with so many countries for the sake of time and memory consumption
df.drop(['Location','WindDir9am','WindDir3pm'], axis=1, inplace=True) 

df = pd.get_dummies(df, drop_first=True, columns = ['WindGustDir'], prefix = ['WindGustDir'])

In [ ]:
def summary(df):
    Types = df.dtypes
    Counts = df.apply(lambda x: x.count())
    Min = df.min()
    Max = df.max()
    Uniques = df.apply(lambda x: x.unique().shape[0]) # .shape[0] yazilmaz ise unique olan degerlerin listelerini getirir.
    Nulls = df.apply(lambda x: x.isnull().sum())

    cols = ['Types', 'Counts', 'Uniques', 'Nulls', 'Min', 'Max']
    str = pd.concat([Types, Counts, Uniques, Nulls, Min, Max], axis = 1, sort=True)

    str.columns = cols
    display(str.sort_values(by='Nulls', ascending=False))
    print('__________Data Types__________\n')
    print(str.Types.value_counts())
summary(df)

In [ ]:
# DecisionTreeRegressor

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor

#drop unnecessary columns and date columns
df_imputation = df.drop(['Date','RainToday','RainTomorrow'], axis=1) 

#define variables to keep the index and the columns
index = df_imputation.index
columns = df_imputation.columns

#imputation steps
imp_tree = IterativeImputer(random_state=0, estimator=DecisionTreeRegressor())
imp_tree.fit(df_imputation)
df_imputed = imp_tree.transform(df_imputation)

#transform imputed data in array format to dataframe
df_imputed_tree = pd.DataFrame(df_imputed, index=index, columns=columns)

df_imputed_tree.isnull().sum()

In [ ]:
df_imputed_tree.info()

In [ ]:
df_imputed_tree.isnull().sum()

# PREDICTION WITH CLASSIFICATION METHODS
# Preparation dependent and independent variables

In [ ]:
df2 = df_imputed_tree.copy()
x_dat = df2.drop(['RainTomorrow_label'],axis=1)
y = df2["RainTomorrow_label"].values

In [ ]:
summary(df2)

# Normalization

In [ ]:
#If there is a outlier values, it must be done before coming here.
x = (x_dat-np.min(x_dat))/(np.max(x_dat)-np.min(x_dat)).values

# LETS TRY CLASSIFICATIONS METHODS
* Now we've got our data split into training and test sets, it's time to build a machine learning model.

* We'll train it (find the patterns) on the training set.

* And we'll test it (use the patterns) on the test set.

* We're going to try machine learning models:

1) Logistic Regression

2) Naive Bayes

3) K-Nearest Neighbor (KNN)

4) Support Vector Mechanism (SVM)

# 1) LOGISTIC REGRESSION

# A) Train-test splitting

In [ ]:
y = df.RainTomorrow_label.values
x_dat = df.drop(["RainTomorrow_label"],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit,GridSearchCV
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=42)

# B) Modeling of Logistic Regression Method

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression() # Default olanlar gelir.C var..
lr_model.fit(x_train,y_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver = 'liblinear')
lr.fit(x_train,y_train)
y_pred = lr.predict(x_test)

In [ ]:
lr #We can see what there is in lr(icinde hangi secenekler vargormek icin).

In [ ]:
lr.intercept_  # Constant coefficient(Sabit katsayi).

In [ ]:
lr.coef_   # Variable coefficients(Degisken katsayilari). # Bu katsayilar denklemin katsayilari. 
           # Ornegin (-) olanlar ters yonde etkiliyor.

# C) Lets control the succes(score) prediction(accuracy_score,confusion m.) on test_data

In [ ]:
l_score = accuracy_score(y_test,y_pred)
l_score
# The y predicted by the y in the test are compared(test deki y ile tahmin edilen yler karsilastiriliyor.
# Dogru tahmin etme yuzdesi bulunuyor)

In [ ]:
c_l = confusion_matrix(y_test,y_pred)# We found the numbers of guessing with confusion matrix.
                                    # (Confusion matrixle tahmin etme sayilarini bulduk.)
c_l                                

In [ ]:
from sklearn.metrics import confusion_matrix   #Hepsi icin yapilabilir.
y_true = y_test
y_pred = lr.predict(x_test)
cmlr = confusion_matrix(y_true, y_pred)
f,ax = plt.subplots(figsize=(6,6))
sns.heatmap(cmlr, annot=True)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

# CLASSICICATION REPORT

In [ ]:
print(classification_report(y_test,y_pred))

# D) LOOK AT ALL PREDICTION VALUE ON TEST DATA

In [ ]:
lr.predict(x_test)[0:10] # Our predictions in the top 10 data test(Ilk 10 datatest deki tahminlerimiz).

In [ ]:
lr.predict_proba(x_test)[0:10] # 1.si 0 olma 2.si 1 olma olasiligi oranlari.

# E) TUNING THE PREDICTION
* We can tune our prediction.

In [ ]:
y_probs = lr.predict_proba(x_test)[:,1]

In [ ]:
y_pred = [1 if i>0.60 else 0 for i in y_probs]
y_pred[-10:]

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
# accuracy_score(y_test,y_pred)
# log_score = accuracy_score(y_test,y_pred)
log_score = accuracy_score(y_test,y_pred)
print ("log score=",log_score)

In [ ]:
# Cross validation (10 katli ) yaparsak
lr_finalscore = cross_val_score(lr_model, x_test, y_test, cv = 10).mean()
lr_finalscore

# 2) NAIVE BAYES METHOD
* In machine learning, Naive Bayes classifiers are a family of simple probabilistic classifiers based on applying Bayes' theorem with strong (naive) independence assumptions between the features.
* Naive Bayes classifiers are highly scalable, requiring a number of parameters linear in the number of variables (features) in a learning problem.

# A) Train-test splitting

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model = nb_model.fit(x_train, y_train)
nb_model

# B) Modeling of Naive Bayes Method

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train, y_train)
y_pred = nb.predict(x_test)
y_pred[:10]

In [ ]:
?nb

# C) Lets control the succes(score) prediction(accuracy_score,confusion m.) on test_data

In [ ]:
n_score = accuracy_score(y_test,y_pred)
n_score

In [ ]:
c_nb = confusion_matrix(y_test,y_pred)
c_nb
# We found the numbers to guess with confusion matrix(confusion matrixle tahmin etme sayilarini bulduk).

# HEATMAP IN CONFUSION MATRIX
* We can see the confusion matrix in Heatmap.

In [ ]:
from sklearn.metrics import confusion_matrix   #Hepsi icin yapilabilir
y_true = y_test
y_pred = nb.predict(x_test)
cmnb = confusion_matrix(y_true, y_pred)
f,ax = plt.subplots(figsize=(6,6))
sns.heatmap(cmnb, annot=True)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

# LOOK AT ALL PREDICTION VALUE ON TEST DATA

In [ ]:
nb.predict(x_test)[0:10] # Our predictions in the top 10 data test(Ilk 10 datatest deki tahminlerimiz).

In [ ]:
nb.predict_proba(x_test)[0:10] #1.si 0 olma 2.si 1 olma olasiligi oranlari

# E) TUNING THE PREDICTION
* WE can tune our prediction

In [ ]:
y_probs = nb.predict_proba(x_test)[:,1]
y_pred = [1 if i>0.60 else 0 for i in y_probs]
y_pred[0:10]

In [ ]:
nb_tuned_bestscore = accuracy_score(y_test,y_pred)
nb_tuned_bestscore

In [ ]:
cmnb_best = confusion_matrix(y_test,y_pred) 
cmnb_best

In [ ]:
# Cross validation (10 katli ) yaparsak
nb_finalscore=cross_val_score(nb_model, x_test, y_test, cv = 10).mean()
nb_finalscore

# 3) KNN METHOD
* In pattern recognition, the k-Nearest Neighbors algorithm (or k-NN for short) is a non-parametric method used for classification and regression.
* A sample is classified by a majority vote of its neighbors, with the sample being assigned to the class most common among its k nearest neighbors (k is a positive integer, typically small).
* In this method we need to choose k value.It means that we chose k number of points of classes which are nearest to the out test point. We can call this small data set. We count the number of classes in the small dataset and determine the highest number of class. Finally we can say our test point belongs to the class.

* While choosing k number we have to be carefull because small k value causes overfitting while big k value causes underfitting.

* Coding is the same for all supervised classes and we jus need to change the last part of the code.

* K=1 SECERSEK OVERFITTING OLABILIR, K= BUYUK SECERSEK UNDERFITTING OLABILIR

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()
knn_model = knn_model.fit(x_train, y_train)
?knn_model

# A) Prediction of KNN

In [ ]:
y_pred = knn_model.predict(x_test)

# B) Accuracy score of KNN

In [ ]:
KNN = accuracy_score(y_test, y_pred)
KNN

In [ ]:
confusion_matrix(y_test,y_pred)

# C) Model Tuning of KNN

In [ ]:
knn_params = {"n_neighbors": np.arange(1,50)}

In [ ]:
knn = KNeighborsClassifier()
knn_cv = GridSearchCV(knn, knn_params, cv=10)
knn_cv.fit(x_train, y_train)

In [ ]:
print("The best score:" + str(knn_cv.best_score_))
print("The best parameters: " + str(knn_cv.best_params_))

In [ ]:
knn = KNeighborsClassifier(4)  # We choose 4 neigboors. Because we get better result than 1, 2 and 3. 
knn_tuned = knn.fit(x_train, y_train)

In [ ]:
knn_finalscore = knn_tuned.score(x_test, y_test)
knn_finalscore

# 4) SVM(SUPPORT VECTOR MACHINES)
* Support Vector Machines are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis. Given a set of training samples, each marked as belonging to one or the other of two categories, an SVM training algorithm builds a model that assigns new test samples to one category or the other, making it a non-probabilistic binary linear classifier.
* SVM is used fo both regression and classification problems, but generally for classification. There is a C parameter inside the SVM algoritma and the default value of C parameter is 1. If C is small, it causes the misclassification. If C is big, it causes ovetfitting. So we need to try C parameter to find best value.
* SVM, hem regresyon hem de sınıflandırma problemleri için kullanılır, ancak genellikle sınıflandırma için kullanılır. SVM içerisinde C parametresi vardır ve C parametresinin default değeri 1'dir. C'nin küçük olması yanlış sınıflandırmaya neden olur. C büyükse overfitting e neden olur. Bu yüzden en iyi değeri bulmak için C parametresini denememiz gerekiyor.

In [ ]:
from sklearn.svm import SVC

svm_model = SVC().fit(x_train,y_train)#we choose default c:1,kernel:'rbf',dagree:3...
?svm_model

# A) Prediction of SVC

In [ ]:
y_pred = svm_model.predict(x_test)

# B) Accuracy score of SVM

In [ ]:
SVC = accuracy_score(y_test,y_pred)
SVC